1. DB를 생성 :
    stockdb
    
2. 테이블을 생성
    - corplist, 상장사 종목 마스터: \
         종목코드, 종목명 \
         key(종목코드) \
         https://han.gl/aFC9t
         
    - ohlv, 날짜별 시세정보: \
         종목코드, 날짜, 종목명, 시가, 고가, 저가, 종가, 거래량 \
         key(종목코드, 날짜)        

In [1]:
import pymysql

db = pymysql.connect(host="localhost", port=3306,
                    user="root", passwd="ehfhtl46")

cursor = db.cursor()

sql = "create database stockdb"

cursor.execute(sql)

cursor.close()
db.commit()

In [2]:
db = pymysql.connect(host="localhost", port=3306,
                    user="root", passwd="ehfhtl46",
                    db="stockdb")

cursor = db.cursor()

sql = """
    create table corplist(
            stock_code varchar(6),
            stock_name varchar(50),
            primary key(stock_code)
            );
    """

cursor.execute(sql)
cursor.close()
db.commit()

In [3]:
db = pymysql.connect(host="localhost", port=3306,
                    user="root", passwd="ehfhtl46",
                    db="stockdb")

cursor = db.cursor()

sql = """
    create table ohlv(
            stock_code    varchar(6)   not null,
            stock_date    date         not null,
            stock_name    varchar(50),
            open_price    int(8),
            high_price    int(8),
            low_price     int(8),
            close_price   int(8),
            volume        int(9),
            primary key(stock_code, stock_date),
            index secondary(stock_date, stock_code)
            );
    """

cursor.execute(sql)
cursor.close()
db.commit()

In [6]:
import requests
import pandas as pd
import time

stock_code = "005930" ; stock_name = "삼성전자" ; page = 1
url = 'https://finance.naver.com/item/sise_day.nhn?'+ 'code=' + str(stock_code) + '&page=' + str(page)

headers = {"user-agent":"Mozilla/5.0"}
resp = requests.get(url, headers=headers)
pd_read = pd.read_html(resp.text)
page_df = pd_read[0].dropna()

page_df["종목코드"]=stock_code; page_df["종목명"]=stock_name
#순서만들기
page_df = page_df[["종목코드","날짜","종목명","시가","고가","저가","종가","거래량"]]
page_df.columns = ["stock_code","stock_date","stock_name","open_price","high_price","low_price","close_price","volume"]
        
page_df

<ipython-input-6-995c93064c4b>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  page_df["종목코드"]=stock_code; page_df["종목명"]=stock_name


,stock_code,stock_date,stock_name,open_price,high_price,low_price,close_price,volume
1,005930,2021.03.23,삼성전자,82600.0,82900.0,81800.0,81800.0,13181232.0
2,005930,2021.03.22,삼성전자,82000.0,82300.0,81700.0,82000.0,12670506.0
3,005930,2021.03.19,삼성전자,82100.0,82500.0,81800.0,81900.0,15869700.0
4,005930,2021.03.18,삼성전자,82800.0,83800.0,82600.0,82900.0,18585244.0
5,005930,2021.03.17,삼성전자,82800.0,82900.0,82000.0,82300.0,11625146.0
9,005930,2021.03.16,삼성전자,82200.0,83000.0,82100.0,82800.0,12293537.0
10,005930,2021.03.15,삼성전자,82800.0,82900.0,81800.0,81800.0,13735798.0
11,005930,2021.03.12,삼성전자,83100.0,83500.0,82400.0,82800.0,13530573.0
12,005930,2021.03.11,삼성전자,81000.0,82500.0,81000.0,82000.0,23818297.0
13,005930,2021.03.10,삼성전자,82400.0,82500.0,80700.0,80900.0,20038422.0


In [10]:
db = pymysql.connect(host="localhost", port=3306,
                    user="root", passwd="ehfhtl46",
                    db="stockdb")

cursor = db.cursor()

In [12]:
for row in range(0, len(page_df)):
    stock_code = page_df.iloc[row,0]
    stock_date= page_df.iloc[row,1]
    stock_name = page_df.iloc[row,2]
    open_price = page_df.iloc[row,3]
    high_price= page_df.iloc[row,4]
    low_price = page_df.iloc[row,5]
    close_price = page_df.iloc[row,6]
    volume = page_df.iloc[row,7]
    sql = "insert into ohlv values ("\
            "'" + str(stock_code) + "','"\
            + str(stock_date) + "','"\
            + str(stock_name) + "',"\
            + str(open_price) + "," + str(high_price) + ","\
            + str(low_price) + "," + str(close_price) + "," + str(volume) + ")"
    print("row =", row, "sql=", sql)
    cursor.execute(sql)

db.commit()
cursor.close()

row = 0 sql= insert into ohlv values ('005930','2021.03.23','삼성전자',82600.0,82900.0,81800.0,81800.0,13181232.0)
row = 1 sql= insert into ohlv values ('005930','2021.03.22','삼성전자',82000.0,82300.0,81700.0,82000.0,12670506.0)
row = 2 sql= insert into ohlv values ('005930','2021.03.19','삼성전자',82100.0,82500.0,81800.0,81900.0,15869700.0)
row = 3 sql= insert into ohlv values ('005930','2021.03.18','삼성전자',82800.0,83800.0,82600.0,82900.0,18585244.0)
row = 4 sql= insert into ohlv values ('005930','2021.03.17','삼성전자',82800.0,82900.0,82000.0,82300.0,11625146.0)
row = 5 sql= insert into ohlv values ('005930','2021.03.16','삼성전자',82200.0,83000.0,82100.0,82800.0,12293537.0)
row = 6 sql= insert into ohlv values ('005930','2021.03.15','삼성전자',82800.0,82900.0,81800.0,81800.0,13735798.0)
row = 7 sql= insert into ohlv values ('005930','2021.03.12','삼성전자',83100.0,83500.0,82400.0,82800.0,13530573.0)
row = 8 sql= insert into ohlv values ('005930','2021.03.11','삼성전자',81000.0,82500.0,81000.0,82000.0,23818297.0)
r